In [ ]:
#Install dependencies
#! pip install SQLAlchemy==1.4.46
#!pip install pandasql

**Name:** Saifullah Khan

**Date**: 9/8/2023

# Customer Insights Report

An ecommerce platform sells different types of products. The sales team is interested in using historical sales data to gain insights about the business. In this notebook, I use SQL to provide answers to the following questions:

1) Which customer placed the largest order of the product Chef Anton's Gumbo Mix?

2) Which salesperson brought the highest revenue in Q4 1996?

3) What is the total revenue from sales to customers in Mexico?

4) Which product has brought the largest revenue in q4 1996?




In [ ]:
import pandas as pd
from google.colab import files
import numpy as np
import datetime as dt
from pandasql import sqldf

First, we'll load the data from our local files.

In [ ]:
def select_sql(df_a):
  return sql("select * from df_a", locals())

In [ ]:
# Upload local Customer.csv table.
uploaded = files.upload()

Saving Customer.csv to Customer.csv


In [ ]:
Customers = pd.read_csv('Customer.csv')
Customers.head()

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
1,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,5021,Mexico
2,3,Antonio Moreno Taquería,Antonio Moreno,Mataderos 2312,México D.F.,5023,Mexico
3,4,Around the Horn,Thomas Hardy,120 Hanover Sq.,London,WA1 1DP,UK
4,5,Berglunds snabbköp,Christina Berglund,Berguvsvägen 8,Luleå,S-958 22,Sweden


In [ ]:
# Upload Order.csv table.
uploaded = files.upload()

Saving Order.csv to Order (1).csv


In [ ]:
Orders = pd.read_csv('Order.csv')
Orders.head()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10248,90,5,1996-07-04,3
1,10249,81,6,1996-07-05,1
2,10250,34,4,1996-07-08,2
3,10251,84,3,1996-07-08,1
4,10252,76,4,1996-07-09,2


In [ ]:
# Upload OrderDetails.csv
ulploaded = files.upload()

Saving OrderDetails.csv to OrderDetails.csv


In [ ]:
OrderDetails = pd.read_csv('OrderDetails.csv')
OrderDetails.head()

,OrderDetailID,OrderID,ProductID,Quantity
0,1,10248,11,12
1,2,10248,42,10
2,3,10248,72,5
3,4,10249,14,9
4,5,10249,51,40


In [ ]:
# Upload Product.csv table
uploaded = files.upload()

Saving Product.csv to Product.csv


In [ ]:
Products = pd.read_csv('Product.csv')
Products.head()

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,1,Chais,1,1,10 boxes x 20 bags,18.00
1,2,Chang,1,1,24 - 12 oz bottles,19.00
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35


# 1) Which customer placed the largest order of the product Chef Anton's Gumbo Mix?


Using the information from the Products table above, we can tell that  Chef Anton's Gumbo Mix has a ProductID = 5. We now can use that information to obtain the customer who ordered the largest quantity of that product in a single order.


In [ ]:
q = '''
SELECT  Orders.CustomerID, OrderDetails.OrderID, OrderDetails.Quantity, OrderDetails.ProductID
FROM Orders
INNER JOIN OrderDetails ON Orders.OrderID = OrderDetails.OrderID
WHERE ProductID = 5
GROUP BY CustomerID
ORDER BY OrderDetails.Quantity DESC
LIMIT 3;
'''

result = sqldf(q)
result.head()

,CustomerID,OrderID,Quantity,ProductID
0,20,10258,65,5
1,15,10290,20,5
2,65,10262,12,5


The query above provides the desired answer: Customer ID 20 was the largest buyer of Chef Anton's Gumbo Mix. We can display just the top customer as well:

In [ ]:
print(f"The customer who ordered the highest quantity of Chef Anton's Gumbo Mix	was CustomerID {result['CustomerID'].values[0]}.")

The customer who ordered the highest quantity of Chef Anton's Gumbo Mix	was CustomerID 20.


#2) Which salesperson brought the highest revenue in Q4 1996?

In [ ]:
q = '''
SELECT Orders.EmployeeID, Orders.OrderDate, SUM(Products.Price * OrderDetails.Quantity) AS total_revenue
FROM Orders
INNER JOIN OrderDetails ON Orders.OrderID = OrderDetails.OrderID
INNER JOIN Products ON OrderDetails.ProductID = Products.ProductID
WHERE OrderDate BETWEEN '1996-10-01' AND '1996-12-31'
GROUP BY EmployeeID
ORDER BY total_revenue DESC
'''

result = sqldf(q)
result.head(3)

,EmployeeID,OrderDate,total_revenue
0,4,1996-10-07,41118.60
1,1,1996-10-09,29871.58
2,5,1996-10-03,23511.30


In [ ]:
print(f"EmployeeID {result['EmployeeID'].values[0]} brought the highest revenue in Q4-1996, bringing a total of USD {result['total_revenue'].values[0]}.")

EmployeeID 4 brought the highest revenue in Q4-1996, bringing a total of USD 41118.59999999999.


# 3) What is the total revenue from sales to customers in Mexico?



In [ ]:
q = '''
WITH OrdersMexico AS (SELECT Orders.OrderID
 FROM Orders
 INNER JOIN Customers ON Customers.CustomerID = Orders.CustomerID
 WHERE Customers.Country="Mexico")

 SELECT SUM(OrderDetails.Quantity * Products.Price)
 FROM OrderDetails
 INNER JOIN Products ON OrderDetails. ProductId = Products.ProductID
 INNER JOIN OrdersMexico
 WHERE OrderDetails.OrderID = OrdersMexico.OrderID
 AND OrderDetails.ProductID = Products.ProductID
 '''

result = sqldf(q)
result.head()

,SUM(OrderDetails.Quantity * Products.Price)
0,5861.56


In [ ]:
print(f"The total revenue from sales to customers in Mexico was USD {round(result['SUM(OrderDetails.Quantity * Products.Price)'].values[0],2)}")

The total revenue from sales to customers in Mexico was USD 5861.56


# 4)Which Product brought the largest revenue in q4 1996?

In [ ]:
q = ''' SELECT Products.ProductName, Orders.OrderDate, SUM(Products.Price * OrderDetails.Quantity) AS total_revenue
FROM Orders
LEFT JOIN OrderDetails ON Orders.OrderID = OrderDetails.OrderID
LEFT JOIN Products On OrderDetails.ProductID = Products.ProductId
WHERE OrderDate BETWEEN '1996-10-01' AND '1996-12-31'
GROUP BY ProductName
ORDER BY total_revenue DESC
LIMIT 3
'''

result = sqldf(q)
result.head()



,ProductName,OrderDate,total_revenue
0,Côte de Blaye,1996-10-15,36890.00
1,Thüringer Rostbratwurst,1996-11-14,9036.67
2,Camembert Pierrot,1996-11-19,8636.00


In [ ]:
print(f"The product name whic has brought the largest revenue in q4-1996 {result['ProductName'].values[0]}.")

The product name whic has brought the largest revenue in q4-1996 Côte de Blaye.
